In [1]:
import pandas as pd

In [2]:
state_correction = {
    'Lousiana' : 'Louisiana',
    'Oklahoma ' : 'Oklahoma',
    'Oaklahoma' : 'Oklahoma',
    'New \nHampshire' : 'New Hampshire',
    'Wyoming ' : 'Wyoming',
    'District of Columbia' : 'DC',
    'Rhode Island ' : 'Rhode Island',
    'Wisconsin ': 'Wisconsin',
}

## Census

In [3]:
census = pd.read_csv('../_CLEAN DATA/pop_estimates_by_state.csv', index_col='id', thousands=',')

In [4]:
census['state'] = census['state'].replace(state_correction)

In [5]:
census_pop_melted = pd.melt(census, id_vars=['state'])

In [6]:
census_pop_melted = census_pop_melted.rename(columns={'variable' : 'date', 'value' : 'est_pop'})

In [7]:
census_pop_melted['date'] = census_pop_melted['date'].astype(int)

In [8]:
census_pop_melted.head()

,state,date,est_pop
0,All states,2004,292805298
1,Alabama,2004,4530729
2,Alaska,2004,659286
3,Arizona,2004,5652404
4,Arkansas,2004,2749686


In [9]:
census_pop_indexed = census_pop_melted.set_index(['state', 'date'])

## Enhanced Penalty

In [10]:
penalty = pd.read_csv('../_CLEAN DATA/statutes_by_state_by_penalty_enhancement.csv')

In [11]:
penalty['state'] = penalty['state'].replace(state_correction)

In [12]:
penalty.head()

,state,penalty_enhancement,race_religion_ethnicity,sexual_orientation,gender,gender_identity,disability,other
0,Alabama,"<a href=""http://codes.lp.findlaw.com/alcode/13...",YES,NO,NO,NO,YES,NO
1,Alaska,"<a href=""http://touchngo.com/lglcntr/akstats/s...",YES,NO,YES,NO,YES,NO
2,Arizona,"<a href=""http://www.azleg.gov/FormatDocument.a...",YES,YES,YES,NO,YES,NO
3,Arkansas,NaN,NO,NO,NO,NO,NO,NO
4,California,"<a href=""http://www.leginfo.ca.gov/cgi-bin/dis...",YES,YES,YES,YES,YES,YES


## Other Statutes

In [13]:
other = pd.read_csv('../_CLEAN DATA/statutes_by_state_other_types.csv')

In [14]:
other['state'] = other['state'].replace(state_correction)

In [15]:
for col in other.columns:
    if col != 'state':
        other[col + '_present'] = other[col].notnull().astype(int)

In [16]:
other_clean = other[['state', 'civil_action_present',
       'data_collection_present', 'police_training_present',
       'institutional_vandalism_present', 'cross_burning_present']]

# Incidents

In [17]:
incidents = pd.read_csv('../_CLEAN DATA/state_hate_crime_totals_final.csv', na_values=['NA', '\xa0'])

In [18]:
incidents['state'] = incidents['state'].str.strip()
incidents['state'] = incidents['state'].replace(to_replace=['Lousiana', 'District of Columbia'], value=['Louisiana', 'DC'])

In [19]:
incidents.state.nunique()

50

In [20]:
def combine_race(x):
    if pd.isnull(x['race']):
        return x['race_ethnicity_ancestry']
    else:
        return x['race'] + x['ethnicity ']

In [21]:
incidents['race_corrected'] = incidents.apply(combine_race, axis=1)

In [22]:
incidents = incidents[[c for c in incidents.columns if c not in ['race', 'race_ethnicity_ancestry', 'ethnicity ']]]

In [23]:
incidents['state'].nunique()

50

In [24]:
incidents_per_capita = pd.merge(incidents, census_pop_melted, on=('state', 'date'), how='left')

In [25]:
incidents_per_capita.head()

,date,state,religion,sexual_orientation,disability,gender,gender_identity,race_corrected,est_pop
0,2015,Alabama,0,2,0,0.0,0.0,8.0,4858979
1,2015,Alaska,0,1,0,0.0,NaN,7.0,738432
2,2015,Arizona,52,57,5,0.0,0.0,162.0,6828065
3,2015,Arkansas,0,2,0,0.0,0.0,3.0,2978204
4,2015,California,191,188,4,1.0,26.0,427.0,39144818


In [26]:
incidents.groupby(['state'])['date'].count().index

Index(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado',
       'Connecticut', 'DC', 'Delaware', 'Florida', 'Georgia', 'Idaho',
       'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana',
       'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota',
       'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada',
       'New Hampshire', 'New Jersey', 'New Mexico', 'New York',
       'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon',
       'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota',
       'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington',
       'West Virginia', 'Wisconsin', 'Wyoming'],
      dtype='object', name='state')

### Join penalty & statute info

In [27]:
penalty_nopen = penalty[[column for column in penalty.columns if column != 'penalty_enhancement']]

In [28]:
penalty_nopen.columns = ['state', 'race_religion_ethnicity_enh_penalty', 'sexual_orientation_enh_penalty', 'gender_enh_penalty',
       'gender_identity_enh_penalty', 'disability_enh_penalty', 'other_enh_penalty']

In [29]:
incidents_all = pd.merge(incidents_per_capita, penalty_nopen, on='state', how='left')

In [30]:
incidents_all_more = pd.merge(incidents_all, other_clean, on='state', how='left')

In [31]:
# weird encoding stuff
incidents_all_more['gender'] = incidents_all_more['gender'].replace('\xa0', '0')

**WARNING:** A hate crime can be tabulated as more than one type, so `all_count` is double counting hate crimes

In [32]:
incidents_all_more['all_count'] = incidents_all_more[['race_corrected', 'religion', 'sexual_orientation', 'disability', 'gender', 'gender_identity']].sum(axis=1)

In [33]:
incidents_all_more.head()

,date,state,religion,sexual_orientation,disability,gender,gender_identity,race_corrected,est_pop,race_religion_ethnicity_enh_penalty,...,gender_enh_penalty,gender_identity_enh_penalty,disability_enh_penalty,other_enh_penalty,civil_action_present,data_collection_present,police_training_present,institutional_vandalism_present,cross_burning_present,all_count
0,2015,Alabama,0,2,0,0.0,0.0,8.0,4858979,YES,...,NO,NO,YES,NO,0,0,0,1,1,10.0
1,2015,Alaska,0,1,0,0.0,NaN,7.0,738432,YES,...,YES,NO,YES,NO,0,0,0,0,0,8.0
2,2015,Arizona,52,57,5,0.0,0.0,162.0,6828065,YES,...,YES,NO,YES,NO,0,1,1,1,1,276.0
3,2015,Arkansas,0,2,0,0.0,0.0,3.0,2978204,NO,...,NO,NO,NO,NO,1,0,0,1,0,5.0
4,2015,California,191,188,4,1.0,26.0,427.0,39144818,YES,...,YES,YES,YES,YES,1,1,1,1,1,837.0


In [34]:
count_columns = ['race_corrected', 'religion', 'sexual_orientation',
                 'disability', 'gender',
                 'gender_identity', 'all_count']

for col in count_columns:
    incidents_all_more[col + '_per_capita'] = incidents_all_more[col] / incidents_all_more['est_pop']

## EXPORT

In [35]:
incidents_all_more.to_csv('../output/incidents_supplemented_2.csv')